In [1]:
import numpy as np
import pandas as pd
from pandas import read_csv

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [32]:
# write your code for task 0 here
data = pd.read_csv("../data/adult.csv")
data = data.replace("?", np.nan)
data.describe()
data.tail(1000)

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
31561,31561,57,Self-emp-not-inc,Doctorate,Married-civ-spouse,Sales,Husband,White,Male,40,United-States,>50K,58
31562,31562,40,Private,HS-grad,Divorced,Transport-moving,Unmarried,White,Male,45,United-States,<=50K,41
31563,31563,36,Private,HS-grad,Married-civ-spouse,Tech-support,Husband,White,Male,50,United-States,<=50K,19
31564,31564,45,Private,Bachelors,Never-married,Adm-clerical,Not-in-family,Black,Female,40,United-States,>50K,295
31565,31565,52,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,<=50K,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173
32558,32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40
32559,32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,38


# Task 1
Print the count of men and women in the dataset.

In [14]:
# write your code for task 1 here
print(data["sex"].value_counts() )

Male      21790
Female    10771
Name: sex, dtype: int64


# Task 2
Find the average age of men in dataset

In [25]:
# write your code for task 2 here
men = data[data["sex"] == "Male"]
res = men["age"].mean()
res

39.43354749885268

# Task 3
Get the percentage of people from Poland (native-country)

In [27]:
# write your code for task 3 here
len_data = len(data)
pol = data[data["native-country"] == "Poland"]
len_pol = len(pol)
res = (len_pol / len_data) * 100
res

0.18426952489174164

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [33]:
# write your code for task 4 here
above_50k = data[data['salary'] == '>50K']
less_50k = data[data['salary'] == '<=50K']
mean_above_50k = above_50k.mean()
std_above_50k = above_50k.std()
mean_less_50k = less_50k.mean()
std_less_50k = less_50k.std()
print(mean_above_50k)
print(std_above_50k)
print(mean_less_50k)
print(std_less_50k)

/var/folders/sv/116kvqy560s82t0cmhb9btgm0000gp/T/ipykernel_76212/1430594308.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_above_50k = above_50k.mean()
/var/folders/sv/116kvqy560s82t0cmhb9btgm0000gp/T/ipykernel_76212/1430594308.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  std_above_50k = above_50k.std()


Unnamed: 0        16370.185691
age                  44.249841
hours-per-week       45.473026
salary K$           200.837521
dtype: float64
Unnamed: 0        9455.124755
age                 10.519028
hours-per-week      11.012971
salary K$           87.085417
dtype: float64
Unnamed: 0        16251.393770
age                  36.783738
hours-per-week       38.840210
salary K$            32.022290
dtype: float64
Unnamed: 0        9382.056122
age                 14.020088
hours-per-week      12.318995
salary K$           10.127874
dtype: float64


/var/folders/sv/116kvqy560s82t0cmhb9btgm0000gp/T/ipykernel_76212/1430594308.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_less_50k = less_50k.mean()
/var/folders/sv/116kvqy560s82t0cmhb9btgm0000gp/T/ipykernel_76212/1430594308.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  std_less_50k = less_50k.std()


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [34]:
# write your code for task 5 here
higher_education = ['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']

result = data[(data['salary'] == '>50K') & (~data['education'].isin(higher_education))]
result


,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
7,7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,307
10,10,37,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,>50K,116
27,27,54,NaN,Some-college,Married-civ-spouse,NaN,Husband,Asian-Pac-Islander,Male,60,South,>50K,275
38,38,31,Private,Some-college,Married-civ-spouse,Sales,Husband,White,Male,38,NaN,>50K,166
55,55,43,Private,Some-college,Married-civ-spouse,Tech-support,Husband,White,Male,40,United-States,>50K,341
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32510,32510,39,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,45,NaN,>50K,212
32518,32518,57,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,116
32519,32519,46,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,48,United-States,>50K,239
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [37]:
# write your code for task 6 here
data.groupby("education")['age'].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,933.0,37.429796,16.720713,17.0,22.00,34.0,52.0,90.0
11th,1175.0,32.355745,15.545485,17.0,18.00,28.0,43.0,90.0
12th,433.0,32.000000,14.334625,17.0,19.00,28.0,41.0,79.0
1st-4th,168.0,46.142857,15.615625,19.0,33.00,46.0,57.0,90.0
5th-6th,333.0,42.885886,15.557285,17.0,29.00,42.0,54.0,84.0
7th-8th,646.0,48.445820,16.092350,17.0,34.25,50.0,61.0,90.0
9th,514.0,41.060311,15.946862,17.0,28.00,39.0,54.0,90.0
Assoc-acdm,1067.0,37.381443,11.095177,19.0,29.00,36.0,44.0,90.0
Assoc-voc,1382.0,38.553546,11.631300,19.0,30.00,37.0,46.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [38]:
# write your code for task 7 here
men = data[data["sex"] == "Male"]
married_men = men[men['marital-status'].str.startswith('Married')]
non_married_men = men[~men['marital-status'].str.startswith('Married')]
married_men_above_50k = (married_men['salary'] == '>50K').mean() * 100
non_married_men_above_50k = (non_married_men['salary'] == '>50K').mean() * 100
print(married_men_above_50k > non_married_men_above_50k)



True


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [44]:
# write your code for task 8 here
data_max = data["hours-per-week"].max()
res = len(data[data['hours-per-week'] == data_max])
res

IndexError: invalid index to scalar variable.

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [45]:
# write your code for task 9 here
data_encoded = pd.get_dummies(data)
correlation_matrix = data_encoded.corr()
threshold = 0.5
high_corr = correlation_matrix[(correlation_matrix >= threshold) & (correlation_matrix != 1)]
